In [26]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path) 
    
from warehouse import DataWarehouse


In [51]:
# Extract property from warehouse
db = DataWarehouse()
property_train = db.query('''
      SELECT CAST(district AS FLOAT), CAST(floorRangeStart AS FLOAT), CAST(floorRangeEnd AS FLOAT), propertyType, area, transactionDate, tenure / 1000000.0000000, resale FROM main__PropertyTransaction
        LIMIT 10
    ''')

price_train = db.query('''
      SELECT price FROM main__PropertyTransaction
        LIMIT 100
    ''')

# property_test = db.query('''
#       SELECT district, floorRangeStart, floorRangeEnd, propertyType, area, transactionDate, tenure, resale FROM main__PropertyTransaction
#         LIMIT 30
#         OFFSET 70

#     ''')

# price_test = db.query('''
#       SELECT price FROM main__PropertyTransaction
#         LIMIT 30
#         OFFSET 70
#     ''')

# Data preprocessing into array
property_train = [[float(d[1]), float(d[2]), float(d[3]), d[4], d[5], d[6], d[7], float(d[8]), d[9], d[10]] for d in property_train]
# property_train = [list(d.values()) for d in property_train]    
property_train = np.asarray(property_train)
print(property_train)

price_train = [list(d.values()) for d in price_train]   
price_train = np.asarray(price_train)
print(price_train)

# property_test = [list(d.values()) for d in property_test]    
# property_test= np.asarray(property_test)
# print(property_test)

# price_test = [list(d.values()) for d in price_test]    
# price_test= np.asarray(price_test)
# print(price_test)

Connecting to 'localhost' with user 'root'
Database | Using database is3107g6.
Database | Query executed successfully.
Database | Query executed successfully.


KeyError: 1

In [31]:
# Build model
def build_mlp_model():

  model = keras.Sequential()
  model.add(Dense(64, activation='relu', input_shape=(8,)))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(1, activation='linear'))
  return model

mlp_model = build_mlp_model()
mlp_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                576       
                                                                 
 dense_7 (Dense)             (None, 64)                4160      
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 4,801
Trainable params: 4,801
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Compile the model
mlp_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])

In [46]:
# def train_model(model, train_epochs, bs, x_train, y_train):

#   return model.fit(
#     x_train, y_train, batch_size=bs, epochs=train_epochs
#   )

# mlp_history = train_model(model = mlp_model, train_epochs = 10, bs = 4, x_train = property_train, y_train = price_train)
mlp_model.fit(property_train, price_train, epochs=5, batch_size=4, validation_split=0.3)



ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).